In [1]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm 

import gpytorch
from gpytorch.kernels import ScaleKernel, MaternKernel, RBFKernel
from gpytorch.priors import GammaPrior
from gpytorch.likelihoods import GaussianLikelihood
import torch
import gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
# Set device and global dtype
device = "cuda:0"
global_dtype = torch.float32

# Ensure reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Add project source path
notebook_dir = os.getcwd()
src_path = os.path.abspath(os.path.join(notebook_dir, '../code'))
if src_path not in sys.path:
    sys.path.append(src_path)

# Import custom modules
from gps import CholeskyGaussianProcess, IterativeGaussianProcess
from util import train, eval, plot_gpr_results, fetch_uci_dataset, memory_dump
from plotting import plot_gp_simple, plot_gp_sample, plot_gp_simple_regions

# Enable autoreloading of modules
%load_ext autoreload
%autoreload 2

In [2]:
train_x, train_y, test_x, test_y = fetch_uci_dataset('protein',r"C:\Users\fredw\chris\Research\softki\data\uci_datasets\uci_datasets\protein\data.csv",train_frac=1/9,val_frac=4/9)
train_x = train_x.to(device)
train_y = train_y.to(device)
test_x = test_x.to(device)

SIZE (45730, 10)
Dataset loaded


In [3]:
kernel = MaternKernel(lengthscale=1.0, variance=1.0)
igp = IterativeGaussianProcess(kernel, noise=0.5, dtype=global_dtype, device=device,
                               cg_tol=1e-2, cg_max_iter=10, warm_start=False, num_probes=0,
                               preconditioner="identity", trace_backend="Hutch",
                               verbose=False, track_iterations=True,pred_lanczos_rank=train_x.shape[0],compute_covariance=False)
alpha = igp.fit(train_x, train_y)
mean = igp.predict(test_x)

In [ ]:
def train(model, train_x, train_y, test_x, test_y, training_iterations=100, lr=0.1):
    # optimizer = torch.optim.Adam([
    #     {'params': model.kernel.parameters()}, 
    #     {'params': model.likelihood.parameters()}
    # ], lr=lr)
    optimizer = torch.optim.Adam([
        {'params': model.kernel.parameters()}, 
        {'params': [model.noise.u]}  # Use raw_value instead of noise()
    ], lr=lr)
    # lr_sched = lambda epoch: 1.0  # Modify this function to change learning rate dynamically
    # scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_sched)

    runtime_log, mll_loss_log, test_rmse_log = [], [], []
    for i in tqdm(range(training_iterations)):
        start_time = time.time()
        optimizer.zero_grad()
        
        model.fit(train_x, train_y)
        loss = model.compute_mll(train_y)
        torch.autograd.set_detect_anomaly(True)
        loss.backward()
        optimizer.step()
        # scheduler.step()  # Update the learning rate

        # print(loss)
        mean, covar = model.predict(test_x)
        total_time = time.time() - start_time
        runtime_log.append(total_time)
        mll_loss_log.append(-loss.item())
        
        test_rmse = (torch.mean(torch.abs(mean - test_y))).item()
        test_rmse_log.append(test_rmse)
        if (i + 1) % 20 == 0:
            print(f'Iter {i+1}/{training_iterations}, Loss: {loss.item():.4f}')
    
    return model, runtime_log, mll_loss_log, test_rmse_log, mean, covar